In [19]:
import json
from collections.abc import Mapping

docs = []

with open('request-docs_results.jsonl', 'r') as f:
    for line in f:
        try:
            response = json.loads(line)[1]["choices"][0]["message"]["content"]
            metadata = json.loads(line)[2]
        except KeyError as ke:
            print("KeyError: ", ke)
            continue
        except IndexError as ie:
            print("IndexError: ", ie)
            continue
        if not isinstance(response, str):
            continue
        if not isinstance(metadata, Mapping):
            continue
        doc = {
            "model": "text-embedding-ada-002", 
            "input": response,
            "metadata": metadata
        }
        docs.append(doc)
        
with open('request-docs-embedding.jsonl', 'w') as f:
    for doc in docs:
        f.write(json.dumps(doc) + '\n')

In [7]:
import json
from collections.abc import Mapping
import pandas as pd
from pprint import pprint
from typing import List

UNIQUE_KEY = "property_id"

df = pd.read_csv('small_df.csv', parse_dates=["date_added"])

with open('request-docs-embedding_results.jsonl', 'r') as f1, open("complete-data.jsonl", "w") as f2:
    for line in f1:
        try:
            request = json.loads(line)[0]["input"]
            response = json.loads(line)[1]["data"][0]["embedding"]
            unique_key = json.loads(line)[2]["row_id"]
        except KeyError as ke:
            print("KeyError: ", ke)
            continue
        except IndexError as ie:
            print("IndexError: ", ie)
            continue
        if not isinstance(response, List):
            continue
        if not isinstance(unique_key, int):
            continue
        if not isinstance(request, str):
            continue
        
        doc = {}
        doc["GPT-Description"] = request
        doc["Ada-Embedding"] = response
        
        data = df[df[UNIQUE_KEY] == unique_key].to_json(orient="records", lines=True, date_format="iso")
        data = json.loads(data)
        doc.update(data)
        
        for key in ("_agency", "agency_id", "_agent", "agent_id", "amenities"):
            doc.pop(key)
        
        f2.write(json.dumps(doc) + '\n')
        


In [1]:
import Database
from Database import CollectionWrapper
import importlib 
importlib.reload(Database)

col = CollectionWrapper()



/Users/konrad/code/fun/nlp-housing/.venv/lib/python3.9/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
import itertools
import json
UNIQUE_KEY = "property_id"
from pprint import pprint
from typing import List

def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))
        
def generate_entry(path):
    with open(path, "r") as f:
        for line in f:
            doc =  json.loads(line)
            rest_of_data = doc.copy()
            for key in ("Ada-Embedding", UNIQUE_KEY):
                rest_of_data.pop(key)
            if not isinstance(doc[UNIQUE_KEY], int):
                continue
            if not isinstance(doc["Ada-Embedding"], list):
                continue
            if len(doc["Ada-Embedding"]) != 1536:
                continue
            result = (
                str(doc[UNIQUE_KEY]),
                doc["Ada-Embedding"],
                rest_of_data
            )
            yield result


async_results = [
    col._index.upsert(vectors=ids_vectors_chunk, async_req=True)
    for ids_vectors_chunk in chunks(generate_entry("/Users/konrad/code/fun/nlp-housing/data/complete-data.jsonl"), 100)
]
# Wait for and retrieve responses (this raises in case of error)
[async_result.get() for async_result in async_results]